In [ ]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

In [ ]:
from json import load as json_load
from random import randint, shuffle
from requests import get
from time import sleep, time
from requests import get
from requests_futures.sessions import FuturesSession
from requests.exceptions import ConnectTimeout, ConnectionError, ProxyError, ReadTimeout
from concurrent.futures import as_completed, Future

from crocket.bittrex.bittrex2 import Bittrex, return_request_input

In [ ]:
PROXY_LIST_PATH = '/home/b3arjuden/crocket/proxy_list.txt'
BITTREX_CREDENTIALS_PATH = '/home/b3arjuden/bittrex_credentials.json'

In [ ]:
def configure_ip(ip):
    
    return {
        'http': ip,
        'https': ip
    }

def process_response(session, response):
    
    try:
        response.data = response.json()
    except:
        response.data = {
               'success': False,
               'message': 'NO_API_RESPONSE',
               'result': None
            }

In [ ]:
# Read files

with open(PROXY_LIST_PATH, 'r') as f:
    proxies = f.read().splitlines()
    
with open(BITTREX_CREDENTIALS_PATH, 'r') as f:
    credentials = json_load(f)

In [ ]:
# Create bittrex objects

b1 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             api_version='v1.1')

b2 = Bittrex(api_key=credentials.get('key'), 
             api_secret=credentials.get('secret'), 
             dispatch=return_request_input,
             api_version='v1.1')

In [ ]:
# Get list of currencies

response = b1.get_markets()

markets = [x.get('MarketName') for x in response.get('result') 
              if x.get('BaseCurrency') == 'BTC' and x.get('IsActive')]

In [ ]:
num_proxies = 200
rind = list(range(num_proxies))

In [ ]:
# Test asynchronous requests

response_dict = {}
working_data = {}

futures = []

with FuturesSession(max_workers=20) as session:

    for ii in range(0, 10): # TODO: replace with while loop

        shuffle(rind)
        start = time()

        for index in range(len(markets)):

            market = markets[index]
            request_input = b2.get_market_history(market)

            proxy = configure_ip(proxies[rind[index]])
            url = request_input.get('url')
            headers = {"apisign": request_input.get('apisign')}
            
            response = session.get(url, 
                                   background_callback=process_response,
                                   headers=headers,
                                   timeout=5,
                                   proxies=proxy)
            
            # Add attributes to response
            response.market = market
            response.url = request_input.get('url')
            response.headers = headers
            
            futures.append(response)

        stop = time()
        time1 = stop - start
        print('Async request time: {}s'.format(str(time1)))

        start = time()
        for future in as_completed(futures):
            
            try:
                response_dict[future.market] = future.result().data
            except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):
                print('Failed API call for {}.'.format(future.market))
                
                while True:
                    
                    print('Retrying...')
                    r = randint(0, num_proxies-1)
                    proxy = configure_ip(proxies[r])
                        
                    try:
                        response = session.get(future.url, 
                                               background_callback=process_response,
                                               headers=future.headers,
                                               timeout=3,
                                               proxies=proxy)
                        response_dict[future.market] = response.result().data
                        break
                        
                    except (ProxyError, ConnectTimeout, ConnectionError, ReadTimeout):
                        print("Future status: {}".format(response))
                        print('Retried API call failed.')
                        pass

                print('Retried API call for {} successful.'.format(future.market))
        
        stop = time()
        time2 = stop - start
        print('Futures execution time: {}s'.format(str(time2)))
        
        start = time()
        working_data, current_datetime = process_data(response_dict, working_data, current_datetime)
        stop = time()
        time3 = stop - start
        print('Processing and uploading data time: {}s'.format(str(time3)))
        
        del futures[:]
        
        if time1 + time2 < 30:
            sleep(30 - (int(time1) + int(time2) + int(time3)))

In [4]:
def process_data(input_data, working_data, current_datetime, interval=60):
    
    if not working_data:
        working_data = input_data
        
    for market in working_data:
        
        working_list = working_data.get(market)
        input_list = input_data.get(market)
        
        last_id = working_list[0].get('Id')
        id_list = [x.get('Id') for x in input_list]
        
        if last_id in id_list:
            overlap_index = id_list.index(last_id)
            working_list = input_list[:overlap_index] + working_list

        else:
            working_list = input_list + working_list
            logger.debug('Latest ID in {} working list not found in input data. '
                         'Adding all input data to working list.')
        
        latest_datetime = convert_bittrex_timestamp_to_datetime(working_list[0].get('TimeStamp'))
        
        if (latest_datetime - current_datetime).total_seconds() > interval:

            start, stop = get_interval_index(working_list, current_datetime, interval)
            logger.debug('START: {}, STOP: {}'.format(str(start), str(stop)))

            if start == stop and (convert_bittrex_timestamp_to_datetime(
                                             working_list[start - 1].get('TimeStamp'))
                                              - current_datetime).total_seconds() <= interval:
                if metrics and len(metrics) > 0:
                    logger.debug('Generating metrics up until latest time.')
                    latest_time = convert_bittrex_timestamp_to_datetime(metrics[0].get('TimeStamp'))
                    while current_datetime < latest_time:
                        new_metrics = calculate_metrics(working_list[start:stop], current_datetime)

                        metrics['volume'] = new_metrics.get('volume')
                        metrics['buyorder'] = new_metrics.get('buyorder')
                        metrics['sellorder'] = new_metrics.get('sellorder')
                        metrics['time'] = new_metrics.get('time')

                        formatted_entry = format_bittrex_entry(metrics)
                        db.insert_query(market, formatted_entry) # TODO: Combine MySQL query
                        current_datetime = current_datetime + timedelta(seconds=interval)
                        logger.debug('Entry added: {}'.format(';'.join(['{}: {}'.format(k, str(v))
                                                                        for k, v in formatted_entry])))
                else:
                    metrics = calculate_metrics(working_list[start:stop], current_datetime)

            else:

                if start == stop:
                    metrics = calculate_metrics([working_list[start - 1]], current_datetime)
                else:
                    metrics = calculate_metrics(working_list[start:stop], current_datetime)

                formatted_entry = format_bittrex_entry(metrics)
                db.insert_query(market, formatted_entry) # TODO: Combine MySQL query
                current_datetime = current_datetime + timedelta(seconds=interval)
                logger.debug('Entry added: {}'.format(';'.join(['{}: {}'.format(k, str(v))
                                                                for k, v in formatted_entry])))

            working_data[market] = working_list[:start]
    
    #TODO: Add MySql query

    return working_data, current_datetime